# 3. [Private 1위 0.6581]  소회의실 Catboost
1등한 코드 따라하기
https://dacon.io/competitions/official/235713/codeshare/2768?page=1&dtype=vote

이상치는 family_size가 7보다 작은 거 삭제  
index제거, FLAG_MOBIL 삭제  
DAYS_EMPLOYED가 양사인 데이터는 현재 부직자로 판단, 0으로 만들기  
DAYS_BRITH, begin_month, DAYS_EMPLOYED 음수값을 절대값 취하기  
파생변수는 잘 모르겠으니 따라하자  
파생변수와 다중공선을 보이는 컬럼 삭제  
스케일링, 인코딩  

In [6]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings, random
warnings.filterwarnings(action='ignore')

from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from category_encoders.ordinal import OrdinalEncoder
from sklearn.model_selection import StratifiedKFold

from sklearn.cluster import KMeans
from catboost import CatBoostClassifier, Pool

In [11]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

## 데이터 전처리

### 1. 결측치처리

In [12]:

train.fillna('NaN', inplace=True)
test.fillna('NaN', inplace=True)

### 2. 이상치처리

In [15]:
train = train[train['family_size'] <= 7]
train = train.reset_index(drop=True)

## Feature engineering

### 1. 의미없는 변수 제거
- index 제거
- 분산이 0인 FLAG_MOBIL 삭제

In [17]:
train.describe()

,index,child_num,income_total,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,family_size,begin_month,credit
count,26451.000000,26451.000000,2.645100e+04,26451.000000,26451.000000,26451.0,26451.000000,26451.000000,26451.000000,26451.000000,26451.000000,26451.000000
mean,13227.047786,0.425920,1.873073e+05,-15958.196628,59082.535859,1.0,0.224680,0.294204,0.091301,2.194208,-26.122793,1.519451
std,7637.264365,0.722095,1.018878e+05,4201.873090,137487.971901,0.0,0.417379,0.455693,0.288042,0.898041,16.558989,0.702325
min,0.000000,0.000000,2.700000e+04,-25152.000000,-15713.000000,1.0,0.000000,0.000000,0.000000,1.000000,-60.000000,0.000000
25%,6612.500000,0.000000,1.215000e+05,-19431.000000,-3153.000000,1.0,0.000000,0.000000,0.000000,2.000000,-39.000000,1.000000
50%,13228.000000,0.000000,1.575000e+05,-15547.000000,-1539.000000,1.0,0.000000,0.000000,0.000000,2.000000,-24.000000,2.000000
75%,19840.500000,1.000000,2.250000e+05,-12446.000000,-407.000000,1.0,0.000000,1.000000,0.000000,3.000000,-12.000000,2.000000
max,26456.000000,5.000000,1.575000e+06,-7705.000000,365243.000000,1.0,1.000000,1.000000,1.000000,7.000000,0.000000,2.000000


In [20]:
train.drop(['index', 'FLAG_MOBIL'], axis=1, inplace=True)
test.drop(['index', 'FLAG_MOBIL'], axis=1, inplace=True)

### 2. DAYS_EMPLOYED

In [40]:
train[train['DAYS_EMPLOYED'] > 0] = 0
test[test['DAYS_EMPLOYED'] > 0] = 0

### DAYS_BIRTH, begin_month, DAYS_EMPLOYED
- 음수값 -> 양수변환

In [50]:
feats = ['DAYS_BIRTH', 'begin_month', 'DAYS_EMPLOYED']
for i in range(len(feats)):
    train[feats[i]] = np.abs(train[feats[i]])

In [51]:
train.describe()

,child_num,income_total,DAYS_BIRTH,DAYS_EMPLOYED,work_phone,phone,email,family_size,begin_month,credit
count,26451.000000,2.645100e+04,26451.000000,26451.000000,26451.000000,26451.00000,26451.000000,26451.000000,26451.000000,26451.000000
mean,0.417980,1.622083e+05,12305.687346,2198.641942,0.224680,0.24604,0.085857,1.902272,21.780538,1.265396
std,0.717874,1.199653e+05,6385.033015,2370.397357,0.417379,0.43071,0.280158,1.201014,17.961032,0.856531
min,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,9.450000e+04,10266.000000,407.000000,0.000000,0.00000,0.000000,1.000000,5.000000,0.000000
50%,0.000000,1.575000e+05,13533.000000,1539.000000,0.000000,0.00000,0.000000,2.000000,19.000000,2.000000
75%,1.000000,2.250000e+05,16816.500000,3153.000000,0.000000,0.00000,0.000000,3.000000,36.000000,2.000000
max,5.000000,1.575000e+06,24611.000000,15713.000000,1.000000,1.00000,1.000000,7.000000,60.000000,2.000000
